In [1]:
from dataclasses import dataclass

from autogen_core import AgentId, MessageContext, RoutedAgent, SingleThreadedAgentRuntime, message_handler
from autogen_core.model_context import BufferedChatCompletionContext
from autogen_core.models import AssistantMessage, ChatCompletionClient, SystemMessage, UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [2]:
@dataclass
class Message:
    content: str

In [3]:
class SimpleAgentWithContext(RoutedAgent):
    def __init__(self, model_client: ChatCompletionClient) -> None:
        super().__init__("A simple agent")
        self._system_messages = [SystemMessage(content="You are a helpful AI assistant.")]
        self._model_client = model_client
        self._model_context = BufferedChatCompletionContext(buffer_size=5)

    @message_handler
    async def handle_user_message(self, message: Message, ctx: MessageContext) -> Message:
        # Prepare input to the chat completion model.
        user_message = UserMessage(content=message.content, source="user")
        # Add message to model context.
        await self._model_context.add_message(user_message)
        # Generate a response.
        response = await self._model_client.create(
            self._system_messages + (await self._model_context.get_messages()),
            cancellation_token=ctx.cancellation_token,
        )
        # Return with the model's response.
        assert isinstance(response.content, str)
        # Add message to model context.
        await self._model_context.add_message(AssistantMessage(content=response.content, source=self.metadata["type"]))
        return Message(content=response.content)

In [4]:
from models import qwen_25_1_5b

runtime = SingleThreadedAgentRuntime()
await SimpleAgentWithContext.register(
    runtime,
    "simple_agent_context",
    lambda: SimpleAgentWithContext(
        # OpenAIChatCompletionClient(
        #     model="gpt-4o-mini",
        #     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY set in the environment.
        # )
        qwen_25_1_5b(),
    ),
)
# Start the runtime processing messages.
runtime.start()
agent_id = AgentId("simple_agent_context", "default")

# First question.
message = Message("Hello, what are some fun things to do in Seattle?")
print(f"Question: {message.content}")
response = await runtime.send_message(message, agent_id)
print(f"Response: {response.content}")
print("-----")

# Second question.
message = Message("What was the first thing you mentioned?")
print(f"Question: {message.content}")
response = await runtime.send_message(message, agent_id)
print(f"Response: {response.content}")

# Stop the runtime processing messages.
await runtime.stop()

Question: Hello, what are some fun things to do in Seattle?
Response: Seattle is an exciting city with so much to offer! Here are some fun activities you can try:

1. Visit Pike Place Market - It's famous because everything inside is handcrafted and handmade!

2. Do a foodie walk like the ones offered by The Portland Collective

3. Look around, the best parts of downtown Seattle aren't as popular or known as it used to be.

4. Go on the water with a cruise tour along Puget Sound.

5. Visit the Alki Beach – It’s the closest beach to downtown and has great views over Elliott Bay.

6. Visit Lake Washington State Park - You can kayak, paddle board or row boat in this stunning place

7. Rent a boat and visit places like Capitola Beach and North Prairie Island. You get great views through trees from these locations

8. Go on a dog sledding trip to see the northern lights – if it’s clear for one of June 25 or August 9 you can go!
-----
Question: What was the first thing you mentioned?
Respons